# Part I

In [8]:
import pyaudio
import numpy as np
CHUNK = 1024
resolution = pyaudio.paInt16 
CHANNELS = 1 
sampling_rate = 16000
record_time=5
Expected_num_of_chunks= int(sampling_rate/CHUNK*record_time) 
p=pyaudio.PyAudio()
stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
frames=[]
for i in range(0,Expected_num_of_chunks): 
    data = stream.read(CHUNK)
    frames.append(data)      
array_frames=np.array(frames)
intframes=np.frombuffer(array_frames, np.int16)
stream.stop_stream()
stream.close()
p.terminate()
import IPython.display as ipd
ipd.Audio(intframes,rate=16000)

# Part II

In [11]:
import keyboard
import time
CHUNK = 1600
resolution = pyaudio.paInt16 
CHANNELS = 1 
sampling_rate = 16000
def hit_to_talk():
    keyboard.wait('s')
    time.sleep(1)
def EnergyPerSampleInDecibel(audioframe):
    audioframe=np.frombuffer(np.array(audioframe),np.int16)
    np_audioframe=np.array(audioframe,dtype='int64')
    energy= 10*np.log10(np.sum(np.square(np_audioframe)))
    return energy
def find_ini_background(audioframe):
    background_energy=0
    for i in range(10):
        background_energy+=EnergyPerSampleInDecibel(audioframe)
    return background_energy/10
def classifyFrame(audioframe):
    forgetfactor=1
    isSpeech=False
    current = EnergyPerSampleInDecibel(audioframe)
    level=current
    threshold=5
    adjustment=0.05
    background=find_ini_background(audioframe)
    if current < background:
        background = current
    else:
        background+=(current - background)*adjustment
    if (level < background): level = background
    if (level - background > threshold): isSpeech = True
    return isSpeech
def record():
    CHUNK = 1600
    resolution = pyaudio.paInt16
    CHANNELS = 1
    sampling_rate = 16000
    stop_count=0
    isSpeech=False
    


In [14]:
import pyaudio
import numpy as np
import wave
p=pyaudio.PyAudio()
stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
while True:
    

391.3018683747959
565.2090586102197
414.7150525210071
461.44542628331743
448.0380678012468
429.55231185474935
407.0444249972946
421.59546438691285
325.8876629372132
387.6275588677879
311.2605001534575
356.2768543016518
346.08978427565484
350.2290527914773
375.2125307297899
335.0829273582968
374.29606702141524
338.38153659804317
337.32798932774966
347.53805931433607
318.18435879447725
360.78837443569904
339.1993072259714
322.3495940962395
307.4084689028244
357.6226137449606
319.7556213153536
328.9142835932332
336.6796383286731
366.60497384805166
351.93028492354296
330.941722577814
316.1966616364075
345.8184435570263
337.76704393343226
332.48994970523137
315.5943017971837
375.2586178740409
350.06480633719116
325.1881454552528
340.2605241919914
312.8076012668715
317.84013415337546
367.0338641127443
338.63205738940457
328.7353772714746
335.85059114902356
321.4313897424399
328.48817146554535
317.8113252314632
432.5679826843425
354.67893516312586
358.2063362911709
385.28458180149977
455.4404

KeyboardInterrupt: 